In [16]:
import featuretools as ft
from dask import bag
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
from datetime import datetime
ft.__version__

'0.12.0'

In [17]:
fulltrain = pd.read_csv("data/finaltrain.csv")

In [18]:
sampletrain = fulltrain.head(100)

In [19]:
train = sampletrain.drop(['click'], axis=1)
y = sampletrain['click']

In [20]:
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self

In [21]:
convert_obj_to_int(train)

,id,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,...,date_int,site_id_int,site_domain_int,site_category_int,app_id_int,app_domain_int,app_category_int,device_id_int,device_ip_int,device_model_int
0,1.000009e+18,1005,0,1,2,15706,320,50,1722,0,...,925657945240155115,-3699807362409133772,7753491774518613300,2963376135392956101,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,-5049776027291409670,-6781995793209040437
1,1.000017e+19,1005,0,1,0,15704,320,50,1722,0,...,925657945240155115,-3699807362409133772,7753491774518613300,2963376135392956101,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,7491899452401661588,4787681255614940163
2,1.000037e+19,1005,0,1,0,15704,320,50,1722,0,...,925657945240155115,-3699807362409133772,7753491774518613300,2963376135392956101,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,6005261213779902421,7586444589573549522
3,1.000064e+19,1005,0,1,0,15706,320,50,1722,0,...,925657945240155115,-3699807362409133772,7753491774518613300,2963376135392956101,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,-3686057402581290185,-2753316137305929290
4,1.000068e+19,1005,1,1,0,18993,320,50,2161,0,...,925657945240155115,7047543187506287775,5068781371477278550,8527761258096310237,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,-6010372456458611121,4801961329533389933
5,1.000072e+19,1005,0,1,0,16920,320,50,1899,0,...,925657945240155115,804056630295342903,7328799648599592352,4864358719125065029,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,-7241379344677485178,7586444589573549522
6,1.000072e+19,1005,0,1,0,20362,320,50,2333,0,...,925657945240155115,-5316826447964788390,-666207768207249098,4864358719125065029,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,4125648523839121244,-9069999996710114881
7,1.000092e+19,1005,1,1,0,20632,320,50,2374,3,...,925657945240155115,-8127499626888949618,5555704963229748459,4864358719125065029,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,-1908853000816696914,7856474898743743948
8,1.000095e+19,1005,0,1,2,15707,320,50,1722,0,...,925657945240155115,-3699807362409133772,7753491774518613300,2963376135392956101,8883529723688211035,7479664941556981152,5546490828158893766,3194416044968131510,578308658391273584,3013916286074337165
9,1.000126e+19,1002,0,0,0,21689,320,50,2496,3,...,925657945240155115,-4414400232705166480,-6318636948241293978,-2909224828499177372,8883529723688211035,7479664941556981152,5546490828158893766,7354729134065089669,4511248598602545410,4612645128314691342


In [22]:
es = ft.EntitySet(id = 'train')

In [23]:
es.entity_from_dataframe(entity_id = 'data', dataframe = train, 
                         index = 'id')

Entityset: train
  Entities:
    data [Rows: 100, Columns: 23]
  Relationships:
    No relationships

In [24]:
feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])

In [25]:
feature_matrix

,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,...,C16 + device_ip_int * device_id_int + device_model_int,device_conn_type + device_id_int * device_ip_int + site_domain_int,C19 + device_type * device_conn_type + site_category_int,app_category_int + C1 * app_id_int + C15,app_id_int + C17 * C1 + device_conn_type,C17 * date_int + site_domain_int,app_category_int + C16 * C20,C1 + C18 * C16 + device_model_int,banner_pos + C18 * C15,app_domain_int + C20 * app_id_int + site_id_int
id,,,,,,,,,,,,,,,,,,,,,
1.000009e+18,1005,0,1,2,15706,320,50,1722,0,35,...,-879166529025559252,-8299559789603783920,-3998923568110889988,-4392764920636036255,-956443995102287461,3633117719862401670,-5546490828158893816,-9057208976261042631,0,-3922658595060035375
1.000017e+19,1005,0,1,0,15704,320,50,1722,0,35,...,-4373182514739589354,-6590378101630666704,-3998923568110890060,-4392764920636036255,-276759368769161359,3633117719862401670,-2881364686808099744,-2980541345178057711,0,-5747203059751368020
1.000037e+19,1005,0,1,0,15704,320,50,1722,0,35,...,5014080354488935096,1414248762623461222,-3998923568110890060,-4392764920636036255,-276759368769161359,3633117719862401670,-2881364686808099744,5871510079372502452,0,-5747203059751368020
1.000064e+19,1005,0,1,0,15706,320,50,1722,0,35,...,6927975996368715084,-6398884571309726446,-3998923568110890060,-4392764920636036255,-276759368769161359,3633117719862401670,-2881364686808099744,-71106936026143800,0,-5747203059751368020
1.000068e+19,1005,1,1,0,18993,320,50,2161,0,35,...,-3614800964869727325,2294862039551961678,-6595243961595208940,-4392764920636036255,-276759368768720164,4369023682268915633,-5546490828158893816,-7075811130845590477,320,8049372127638095686
1.000072e+19,1005,0,1,0,16920,320,50,1899,0,431,...,-6343462221940424256,-8907027100608930044,-1525857740860791696,-4392764920636036255,-276759368768983474,-4517491951438433767,-4813312336501253224,5871510079372502452,0,-1336073252217330326
1.000072e+19,1005,0,1,0,20362,320,50,2333,0,39,...,-874195389195380250,8370517442796494284,-8339836045802466616,-4392764920636036255,-276759368768547304,-3445291414645465667,-5546490828158893816,-2658424281146906851,0,-4563242779223746965
1.000092e+19,1005,1,1,0,20632,320,50,2374,3,39,...,-815568447012039744,-6139324439539948602,-8339836045802466616,-4392764920636036255,-276759368768506099,2170987233785036932,-5546490828158893816,5673490312296306720,1280,-1760014804382837833
1.000095e+19,1005,0,1,2,15707,320,50,1722,0,35,...,2995750672716619878,7289197703014111712,-3998923568110889988,-4392764920636036255,-956443995102287461,3633117719862401670,-5546490828158893816,3719839416342436051,0,-3922658595060035375


In [26]:
X = feature_matrix.copy()
X = X.reset_index()
X = X.replace([np.inf, -np.inf], np.nan)

In [27]:
clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [28]:
scores = cross_val_score(estimator=clf, X=X, y=y, cv=5,
                         scoring="roc_auc", verbose=True)
print("AUC {:.4f} +/- {:.4f}".format(scores.mean(), scores.std()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.6687 +/- 0.1323


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   20.0s finished


In [29]:
n_estimators = [1, 3, 5, 10, 25, 50, 100, 250, 500]
max_depth = [1, 3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
scores_list = []

In [30]:
for est in n_estimators:
    for depth in max_depth:
        clf = RandomForestClassifier(
            n_estimators=est, max_depth=depth, n_jobs=-1)
        scores = cross_val_score(
            clf,
            X=X,
            y=y,
            scoring='roc_auc',
            cv=10,
            n_jobs=-1
        )
        scores_list.append([est, depth, scores.mean(), scores.std()])
top_scores_df = pd.DataFrame(scores_list, columns=[
                             'n_estimators', 'max_depth', 'mean', 'std'])

In [31]:
top_scores_df

,n_estimators,max_depth,mean,std
0,1,1.0,0.475000,0.041458
1,1,3.0,0.543750,0.125156
2,1,5.0,0.481250,0.153221
3,1,10.0,0.487500,0.203101
4,1,20.0,0.537500,0.145774
5,1,30.0,0.618750,0.237911
6,1,40.0,0.493750,0.106250
7,1,50.0,0.543750,0.225434
8,1,60.0,0.556250,0.204347
9,1,70.0,0.543750,0.268168


In [ ]:
top_features = utils.feature_importances(clf, features_encoded, n=20)

In [ ]:
ft.save_features(top_features, "top_features")